In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline

import pandas, seaborn, numpy, collections, sefara, varcode, uncertainties, numpy, statsmodels, pickle, scipy
%matplotlib inline
from matplotlib import pyplot
import topiary
import mhctools
import matplotlib_venn
import intervaltree
import statsmodels.robust
import mhcflurry
from varcode.common import memoize
pandas.set_option('display.max_columns', None)
import scipy, sys, sklearn
import sklearn.utils
import matplotlib
#import common as c
import sklearn.tree
import pymc3
from scipy import optimize
import glob


def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
matplotlib.rc('text', usetex=False)
#reload(c)

%matplotlib inline
%config InlineBackend.figure_format = 'png'

import pystan
import pyfaidx



/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [5]:
df = pandas.read_csv("../data/all_signatures.csv", index_col=0).T
df

Signature 1   Signature 2  Signature 3  Signature 4  Signature 5  \
A[C>A]A     0.011098  6.827082e-04     0.022172       0.0365     0.014942   
A[C>A]C     0.009149  6.191072e-04     0.017872       0.0309     0.008961   
A[C>A]G     0.001490  9.927896e-05     0.002138       0.0183     0.002208   
A[C>A]T     0.006234  3.238914e-04     0.016265       0.0243     0.009207   
A[C>G]A     0.001801  2.634810e-04     0.024003       0.0097     0.011671   
A[C>G]C     0.002581  2.698660e-04     0.012160       0.0054     0.007292   
A[C>G]G     0.000593  2.192339e-04     0.005275       0.0031     0.002304   
A[C>G]T     0.002964  6.109735e-04     0.023278       0.0054     0.011696   
A[C>T]A     0.029515  7.441557e-03     0.017872       0.0120     0.021839   
A[C>T]C     0.014323  2.726312e-03     0.008896       0.0075     0.012756   
A[C>T]G     0.171647  3.322083e-03     0.003573       0.0028     0.016760   
A[C>T]T     0.012624  3.326528e-03     0.014798       0.0059     0.016478   
A[T>A]A     0.004022  1.323770e-07     0.008429       0.0048     0.008903   
A[T>A]C     0.002371  1.130696e-04     0.007373       0.0039     0.007399   
A[T>A]G     0.002811  5.329294e-04     0.007357       0.0100     0.011508   
A[T>A]T     0.008361  1.491110e-04     0.008754       0.0030     0.011194   
A[T>C]A     0.013916  1.303510e-03     0.013036       0.0084     0.035367   
A[T>C]C     0.006275  4.255767e-04     0.009186       0.0020     0.013771   
A[T>C]G     0.010138  5.754730e-04     0.011717       0.0081     0.028449   
A[T>C]T     0.009256  1.488164e-03     0.016979       0.0036     0.027303   
A[T>G]A     0.001588  3.371841e-05     0.002351       0.0000     0.003462   
A[T>G]C     0.001784  2.484483e-05     0.001464       0.0002     0.002247   
A[T>G]G     0.001386  2.734541e-04     0.009054       0.0015     0.005490   
A[T>G]T     0.003159  2.176949e-04     0.007031       0.0002     0.003821   
C[C>A]A     0.006596  6.774450e-04     0.018782       0.0461     0.009675   
C[C>A]C     0.007342  2.136810e-04     0.015760       0.0614     0.004952   
C[C>A]G     0.000893  6.770460e-06     0.001963       0.0088     0.002801   
C[C>A]T     0.007187  4.163329e-04     0.014723       0.0432     0.011013   
C[C>G]A     0.001285  2.777208e-05     0.016833       0.0105     0.007538   
C[C>G]C     0.000702  2.796439e-04     0.013531       0.0097     0.007633   
...              ...           ...          ...          ...          ...   
G[T>C]G     0.004978  4.950440e-07     0.006099       0.0042     0.014104   
G[T>C]T     0.010667  5.785981e-04     0.007509       0.0024     0.015667   
G[T>G]A     0.000099  9.555239e-05     0.004144       0.0000     0.001628   
G[T>G]C     0.000202  4.700238e-05     0.004502       0.0000     0.000328   
G[T>G]G     0.001188  1.099257e-04     0.016391       0.0018     0.005949   
G[T>G]T     0.000801  8.647718e-05     0.007067       0.0002     0.003307   
T[C>A]A     0.012250  1.512743e-02     0.011653       0.0330     0.014774   
T[C>A]C     0.011162  6.532492e-03     0.016607       0.0538     0.012043   
T[C>A]G     0.002275  1.656455e-03     0.001357       0.0104     0.003902   
T[C>A]T     0.015259  1.239461e-02     0.016328       0.0370     0.018243   
T[C>G]A     0.001875  3.724185e-02     0.016041       0.0032     0.002681   
T[C>G]C     0.002067  1.941341e-05     0.020150       0.0105     0.007924   
T[C>G]G     0.000305  1.625465e-03     0.002528       0.0031     0.001319   
T[C>G]T     0.003152  6.687990e-02     0.032674       0.0050     0.006645   
T[C>T]A     0.014492  4.199414e-01     0.008880       0.0035     0.010998   
T[C>T]C     0.017681  8.197250e-02     0.013530       0.0070     0.020645   
T[C>T]G     0.076002  4.772019e-02     0.001705       0.0011     0.007534   
T[C>T]T     0.013762  2.286749e-01     0.010304       0.0077     0.011787   
T[T>A]A     0.005600  8.075249e-05     0.007133       0.0045     0.009206   
T[T>A]C     0.001999  4.802898e-06     0.009103       0.0046     0.006835   
T[T>A

In [6]:
signatures_to_plot = [x for x in df.columns if 'Signature' not in x]
signatures_to_plot

['Cisplatin: Gallus gallus',
 'Cyclophosphamide: Gallus gallus',
 'Etoposide: Gallus gallus',
 'Cisplatin: C. Elegans wildtype',
 'Cisplatin: C. Elegans fan-1',
 'Cisplatin: C. Elegans fcd-2',
 'Cisplatin: C. Elegans lig-4',
 'Cisplatin: C. Elegans lig-4; fcd-2',
 'Cisplatin: C. Elegans mus-81',
 'Cisplatin: C. Elegans polq-1',
 'Cisplatin: C. Elegans slx-1',
 'Cisplatin: C. Elegans xpf-1']

In [75]:
seaborn.set_context('paper')
seaborn.set_style("white")
pyplot.figure(figsize=(6,8))

grid = (len(signatures_to_plot), 6)

prev_axes = []

for (row, signature) in enumerate(signatures_to_plot):
    for (col, kind) in enumerate(sorted(set(df.index.str.slice(2,5)))):
        sharex = sharey = None
        if col > 1:
            sharey = prev_axes[-1]
        if row != 0:
            sharex = prev_axes[-grid[1]]

        ax = pyplot.subplot2grid(grid, (row, col), sharex=sharex, sharey=sharey)
        if col == 0:
            pyplot.text(0, 0.32, signature, fontsize=6)

        prev_axes.append(ax)

        sub_df = df[df.index.str.contains(kind)].copy()
        sub_df["mutation"] = sub_df.index
        
        pyplot.bar(numpy.arange(sub_df.shape[0]),
                   sub_df[signature].values,
                   color='black')

        ax.tick_params(axis='x', which='major', pad=2)
        pyplot.xticks(
            numpy.arange(sub_df.shape[0]) + 0.5, [x.replace("[%s]" % kind, r"$\bullet$") for x in sub_df.index],
            fontsize=5,
        )
        pyplot.xticks(rotation=90)
        pyplot.ylabel("")
        pyplot.xlabel("")
        ax.text(0.5,.70,
            kind.replace(">", "$\\rightarrow$"),
            bbox=dict(facecolor='white', edgecolor='black', pad=0.2),
            fontsize=8,
            horizontalalignment='center',
            transform=ax.transAxes)
        pyplot.ylim(ymin=0, ymax=0.30)
        ax.yaxis.grid('on', linewidth=1)

        #ax.label_outer()
        pyplot.yticks([0.0, 0.05, 0.1, 0.15, 0.20, 0.25], fontsize=5)
        for label in ax.get_yticklabels():
            label.set_visible(ax.is_first_col())

pyplot.tight_layout(w_pad=0, h_pad=0.85)

pyplot.savefig("../figures/extracted_signatures.pdf")

In [61]:
ax.yaxis.grid?